# 🚀 Slack to Jira Workflow Agent

[![MCP Hackathon](https://img.shields.io/badge/MCP-Hackathon-blue)](https://github.com/modelcontextprotocol)
[![Python](https://img.shields.io/badge/Python-3.8+-green)](https://python.org)
[![Llama Stack](https://img.shields.io/badge/Llama-Stack-orange)](https://github.com/meta-llama/llama-stack)

This notebook demonstrates how to build an intelligent workflow agent that seamlessly bridges Slack discussions and Jira issue tracking using **Model Context Protocol (MCP)** tools and **Llama Stack**.

## 🎯 What This Agent Does

The agent performs an end-to-end workflow:

> **"Analyze a Slack discussion thread, extract key information about technical issues or feature requests, create well-structured Jira stories with appropriate fields, and automatically link them to specified epics."**

### ✨ Key Features

- **🧠 Intelligent Analysis**: Extracts technical issues, bug reports, and feature requests from Slack conversations
- **📝 Professional Formatting**: Creates well-structured Jira stories with proper fields and descriptions
- **🔗 Epic Linking**: Automatically links stories to appropriate epics for better project organization
- **🔄 Context Preservation**: Maintains discussion context and technical details in Jira descriptions
- **⚡ Automated Workflow**: End-to-end process from Slack discussion to Jira story creation

## 🛠️ MCP Tools Integration

### Slack MCP Server
- **Purpose**: Retrieve discussion content from Slack channels and threads
- **Repository**: [Slack MCP Server](https://github.com/modelcontextprotocol/servers-archived/tree/main/src/slack)
- **Capabilities**: Read messages, threads, channel content, and user information

### Jira MCP Server  
- **Purpose**: Create and manage Jira issues, including linking stories to epics
- **Repository**: [Jira MCP Server](https://github.com/sooperset/mcp-atlassian)
- **Capabilities**: Create issues, update fields, link to epics, manage projects

## 🏗️ Architecture Overview

```mermaid
graph LR
    A[Slack Discussion] --> B[Llama Stack Agent]
    B --> C[Slack MCP Server]
    B --> D[Jira MCP Server]
    C --> E[Extract Messages]
    D --> F[Create Jira Story]
    E --> G[Analyze Content]
    G --> H[Generate Summary]
    H --> F
    F --> I[Link to Epic]
```

This notebook will guide you through building and running this intelligent workflow system.

## 📋 Prerequisites

Before running this notebook, ensure you have:

### 🔧 Technical Requirements
- **Python 3.8+** installed on your system
- **Llama Stack server** running and accessible
- **Slack MCP server** configured with appropriate permissions
- **Jira MCP server** configured with issue creation permissions

### 🔑 Access Requirements
- **Slack Workspace**: Read access to channels and threads
- **Jira Instance**: Create issues and link to epics permissions
- **Environment Configuration**: Properly configured `.env` file

### 🚀 Quick Setup

If you haven't set up the environment yet, run the automated setup:

```bash
# Clone the repository
git clone https://github.com/your-username/Slack2Jira.git
cd Slack2Jira

# Run automated setup
./setup.sh

# Configure your environment
cp env.example .env
# Edit .env with your actual configuration values
```

### 📝 Configuration Required

Update your `.env` file with:
- `REMOTE_BASE_URL`: Your Llama Stack server URL
- `REMOTE_SLACK_MCP_URL`: Slack MCP server endpoint  
- `REMOTE_JIRA_MCP_URL`: Jira MCP server endpoint
- `SLACK_CHANNEL_ID`: Target Slack channel ID
- `SLACK_THREAD_TS`: Specific thread timestamp
- `JIRA_EPIC_KEY`: Epic to link stories to
- `JIRA_PROJECT_KEY`: Target Jira project

### ✅ Verification

Before proceeding, verify:
- [ ] All servers are running and accessible
- [ ] Environment variables are properly configured
- [ ] You have necessary permissions for both Slack and Jira
- [ ] Dependencies are installed (`pip install -r requirements.txt`)

In [ ]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient
from llama_stack_client import Agent
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
from llama_stack_client.lib.agents.event_logger import EventLogger

# pretty print of the results returned from the model/agent
from termcolor import cprint
import sys
import json
sys.path.append('..')  
from src.utils import step_printer

base_url = os.getenv("REMOTE_BASE_URL")

# Tavily search API key is required for some of our demos and must be provided to the client upon initialization.
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}

client = LlamaStackClient(
    base_url=base_url,
    provider_data=provider_data
)

print(f"Connected to Llama Stack server")

# model_id for the model you wish to use that is configured with the Llama Stack server
model_id = "granite32-8b"

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = 512

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")


## 🔧 MCP Tool Registration & Validation

### Why Tool Registration Matters

When a Llama Stack instance is redeployed, MCP tools may need to be re-registered. Additionally, attempting to register a tool with an existing `toolgroup_id` will result in an error.

### Automatic Registration Process

The following code will:

1. **Check existing tools**: Validate that both `mcp::slack` and `mcp::jira` tools are registered
2. **Auto-register missing tools**: Register any missing tools using their configured endpoints
3. **Prevent conflicts**: Avoid duplicate registrations that would cause errors

### Tool Endpoints

- **Slack MCP**: Configured via `REMOTE_SLACK_MCP_URL` environment variable
- **Jira MCP**: Configured via `REMOTE_JIRA_MCP_URL` environment variable

This ensures our agent has access to all necessary tools for the Slack-to-Jira workflow.

In [ ]:
slack_mcp_url = os.getenv("REMOTE_SLACK_MCP_URL")
jira_mcp_url = os.getenv("REMOTE_JIRA_MCP_URL")

registered_tools = client.tools.list()
registered_toolgroups = [t.toolgroup_id for t in registered_tools]

if "mcp::slack" not in registered_toolgroups:
    client.toolgroups.register(
        toolgroup_id="mcp::slack",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":slack_mcp_url},
   )

if "mcp::jira" not in registered_toolgroups:
    client.toolgroups.register(
        toolgroup_id="mcp::jira",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":jira_mcp_url},
    )

print(f"Your Llama Stack server is registered with the following tool groups @ {set(registered_toolgroups)} \n")

## 🤖 Agent Configuration - Slack to Jira Workflow

### Agent Specialization

Our agent is specifically designed to excel at analyzing Slack discussions and creating professional Jira stories. The agent's prompt engineering focuses on:

### 🎯 Core Capabilities

1. **📊 Discussion Analysis**: Extract technical issues, bug reports, and feature requests
2. **🔍 Detail Identification**: Capture problem descriptions, reproduction steps, and error messages  
3. **📝 Professional Writing**: Generate clear, concise summaries and detailed descriptions
4. **🔗 Epic Integration**: Properly link stories to specified epics for project organization
5. **🏗️ Context Preservation**: Maintain discussion context and technical nuances

### 🧠 Agent Behavior

The agent is programmed to be thorough and professional, ensuring that:
- All relevant technical details are captured
- Jira stories follow best practices for structure and content
- Discussion context is preserved for future reference
- Epic relationships are properly established

In [ ]:
model_prompt = """You are a helpful assistant specialized in analyzing Slack discussions and creating well-structured Jira stories.

When analyzing Slack discussions, you should:
1. Extract the main technical issue, bug report, or feature request from the conversation
2. Identify key details like:
   - Problem description
   - Steps to reproduce (if mentioned)
   - Expected vs actual results (if discussed)
   - Error messages or symptoms
   - Proposed solutions or workarounds
3. Create a clear, concise summary for the Jira story title
4. Write a detailed description that includes context from the Slack discussion
5. Link the story to the appropriate epic when specified

Always be thorough in your analysis and create professional, well-formatted Jira stories that capture the essence of the Slack discussion.
"""

## 🚀 Workflow Execution - Slack Discussion to Jira Story

### 🔄 End-to-End Process

Our intelligent agent will perform a comprehensive workflow that transforms Slack discussions into professional Jira stories:

### 📋 Workflow Steps

1. **📥 Retrieve Messages**: Fetch the complete Slack thread discussion with all replies and context
2. **🧠 Intelligent Analysis**: Analyze the conversation to identify technical issues, bugs, or feature requests  
3. **📊 Epic Context**: Gather information about the target epic for proper project alignment
4. **📝 Story Creation**: Generate a well-structured Jira story with professional formatting
5. **🔗 Epic Linking**: Automatically link the new story to the specified epic

### ⚙️ Configuration

The workflow uses environment variables for flexible configuration:
- **Slack Channel**: Target channel for message retrieval
- **Thread Timestamp**: Specific discussion thread to analyze
- **Jira Project**: Destination project for story creation
- **Epic Key**: Parent epic for story linking

### 🎯 Expected Outcomes

After execution, you'll have:
- A professionally formatted Jira story
- Complete context preservation from Slack discussion
- Proper epic relationship established
- All technical details captured and organized

In [ ]:
# Configuration for the specific Slack discussion and Jira epic
SLACK_CHANNEL_ID = "C08UPFE3VA5"
SLACK_THREAD_TS = "1748436131.235159"
JIRA_EPIC_KEY = "ILAICOMUTY-6"
JIRA_PROJECT_KEY = "ILAICOMUTY"

# Create agent with Slack and Jira tools
agent = ReActAgent(
    client=client,
    model=model_id,
    tools=["mcp::slack", "mcp::jira"],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
    sampling_params={"max_tokens": 1024},
)

# Define the workflow prompt
workflow_prompt = f"""
Please perform the following workflow:

1. First, retrieve the Slack thread messages from channel {SLACK_CHANNEL_ID} with thread timestamp {SLACK_THREAD_TS}
2. Analyze the discussion to understand the main technical issue or topic being discussed
3. Get details about the epic {JIRA_EPIC_KEY} to understand the context
4. Create a new Jira story in project {JIRA_PROJECT_KEY} with:
   - A clear, descriptive summary based on the Slack discussion
   - A detailed description that includes:
     * Summary of the Slack discussion
     * Key technical details mentioned
     * Any error messages or symptoms described
     * Steps to reproduce (if mentioned)
     * Expected vs actual results (if discussed)
   - Link it to epic {JIRA_EPIC_KEY} using the parent field
   - Set appropriate issue type as "Story"

Make sure to create a professional, well-structured Jira story that captures the essence of the Slack discussion.
"""

session_id = agent.create_session("slack-to-jira-workflow")

print("\n" + "="*80)
cprint(f"Starting Slack to Jira Workflow", "blue")
print("="*80)

response = agent.create_turn(
    messages=[
        {
            "role": "user",
            "content": workflow_prompt,
        }
    ],
    session_id=session_id,
    stream=stream
)

if stream:
    for log in EventLogger().log(response):
        log.print()
else:
    step_printer(response.steps)

## 🎉 Key Takeaways & Success Metrics

### 🏆 What We Accomplished

This notebook successfully demonstrated building a sophisticated workflow agent that bridges Slack and Jira using MCP tools with Llama Stack:

#### ✅ Core Achievements

1. **🔗 Seamless Integration**: Connected two different platforms (Slack and Jira) through a unified agent interface
2. **🔄 Context Preservation**: Maintained discussion context when creating Jira stories from Slack threads  
3. **🧠 Intelligent Analysis**: Used LLM capabilities to extract meaningful information from unstructured conversations
4. **⚡ Automated Workflow**: Created an end-to-end process that reduces manual effort in issue tracking
5. **🏗️ Flexible Architecture**: Built a reusable framework adaptable for different use cases

### 🚀 Future Enhancement Opportunities

This foundation can be extended to support more sophisticated workflows:

#### 🎯 Immediate Extensions
- **📊 Priority Detection**: Automatic priority and severity assignment based on discussion content
- **🔍 Duplicate Prevention**: Check for similar existing Jira issues before creation
- **📎 Attachment Support**: Handle files, images, and logs from Slack discussions
- **👥 Smart Assignment**: Intelligent routing based on discussion participants and content

#### 🌟 Advanced Integrations  
- **🔄 Multi-platform Sync**: Bidirectional synchronization between Slack and Jira
- **🐙 GitHub Integration**: Link code repositories and pull requests
- **🎫 ServiceNow Support**: Enterprise service management integration
- **📈 Analytics Dashboard**: Workflow metrics and performance tracking

### 💡 Technical Innovation

The combination of **MCP tools** and **Llama Stack** provides a powerful foundation for:
- **Intelligent Automation**: AI-driven workflow orchestration
- **Platform Agnostic**: Universal integration capabilities
- **Context Awareness**: Maintaining semantic understanding across platforms
- **Scalable Architecture**: Enterprise-ready automation solutions

### 🎯 Business Impact

This solution enhances team productivity by:
- **⏰ Time Savings**: Eliminating manual ticket creation processes
- **📝 Quality Improvement**: Ensuring consistent, professional issue documentation
- **🔍 Context Retention**: Preventing information loss during platform transitions
- **📊 Process Standardization**: Establishing repeatable workflow patterns

---

## 🌟 Ready for Production?

This notebook provides a solid foundation for production deployment. Consider these next steps:

1. **🔧 Configuration Management**: Implement robust environment configuration
2. **🛡️ Error Handling**: Add comprehensive error handling and retry logic
3. **📊 Monitoring**: Implement logging and metrics collection
4. **🧪 Testing**: Develop comprehensive test suites
5. **📚 Documentation**: Create user guides and API documentation

**The future of intelligent workflow automation starts here!** 🚀